<a href="https://colab.research.google.com/github/Asishcharan/-Mental-Health-Chat-Analysis-/blob/main/CVD_Detection_Early.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# ----- PARAMETERS -----
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 50  # train longer for better convergence

# ----- DATA LOADING -----
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale"
)

# ----- DATA AUGMENTATION (safe) -----
data_augmentation = tf.keras.Sequential([
    layers.RandomBrightness(factor=0.1),  # mild brightness
    layers.RandomContrast(factor=0.1)
])

# ----- NORMALIZATION -----
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# ----- PERFORMANCE OPTIMIZATION -----
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ----- CNN MODEL FROM SCRATCH -----
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# ----- COMPILE -----
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ----- CALLBACKS -----
checkpoint_cb = callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/ecg_model_best_scratch.keras",
    save_best_only=True
)
earlystop_cb = callbacks.EarlyStopping(
    patience=7,
    restore_best_weights=True
)
lr_reduce_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# ----- TRAIN -----
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb, lr_reduce_cb]
)

# ----- EVALUATE -----
test_loss, test_acc = model.evaluate(test_ds)
print("Final Test Accuracy:", test_acc)

# ----- SAVE FINAL MODEL -----
model.save("/content/drive/MyDrive/ecg_model_final_scratch.keras")


Found 3023 files belonging to 4 classes.
Found 928 files belonging to 4 classes.


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 124, 124, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 124, 124, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 60, 60, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 58, 58, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 58, 58, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 27, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 25, 25, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     4,718,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │         1,02

 Total params: 5,008,100 (19.10 MB)

 Trainable params: 5,007,204 (19.10 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 90s 363ms/step - accuracy: 0.2623 - loss: 2.5062 - val_accuracy: 0.1853 - val_loss: 1.9945 - learning_rate: 1.0000e-04
Epoch 2/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.3121 - loss: 1.3914 - val_accuracy: 0.1853 - val_loss: 2.3878 - learning_rate: 1.0000e-04
Epoch 3/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.2936 - loss: 1.3771 - val_accuracy: 0.1853 - val_loss: 2.1956 - learning_rate: 1.0000e-04
Epoch 4/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.3227 - loss: 1.3683 - val_accuracy: 0.1832 - val_loss: 1.6410 - learning_rate: 1.0000e-04
Epoch 5/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.3119 - loss: 1.3646 - val_accuracy: 0.3341 - val_loss: 1.3323 - learning_rate: 1.0000e-04
Epoch 6/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - accuracy: 0.3424 - loss: 1.3489 - val_accuracy: 0.4450 - val_loss: 1.3193 - learning_rate: 1.0000e-04
Epoch 7/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.383

In [10]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred = np.concatenate([np.argmax(model.predict(x), axis=1) for x, y in test_ds], axis=0)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

# Now define validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/test",  # path to your test/val folder
    image_size=(128,128),
    batch_size=32
)

# Normalize
normalization_layer = layers.Rescaling(1./255)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 928 files belonging to 4 classes.


In [21]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from tensorflow.keras import layers
import numpy as np

# Load your trained CNN
cnn_model = load_model("/content/drive/MyDrive/ecg_model_final_scratch.keras")
print("CNN model loaded!")


CNN model loaded!


In [22]:
 # Grayscale images to match your trained model
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/train",
    image_size=(128,128),
    batch_size=32,
    color_mode="grayscale"  # important!
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/test",
    image_size=(128,128),
    batch_size=32,
    color_mode="grayscale"
)

# Normalize images
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 3023 files belonging to 4 classes.
Found 928 files belonging to 4 classes.


In [24]:
import tensorflow as tf
from tensorflow.keras import layers

train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/ECG_DATA/train",
    image_size=(128,128),
    batch_size=32,
    color_mode="grayscale"  # important!
)

normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


Found 3023 files belonging to 4 classes.


In [25]:
 for x_batch, y_batch in train_ds.take(1):
    _ = cnn_model(x_batch)  # now input/output shapes are defined


In [26]:
from tensorflow.keras import Model

feature_layer = cnn_model.layers[-3]  # layer before final Dense/softmax
feature_extractor = Model(inputs=cnn_model.input, outputs=feature_layer.output)
print("Feature extractor ready!")


AttributeError: The layer sequential_12 has never been called and thus has no defined input.

In [13]:
import os

path = "/content/drive/MyDrive/"
os.listdir(path)


["I am sharing 'Asish Asish' with you",
 'peddamma.mp4',
 'VTU21511_Academic Fee - Summer Semester - 2024-2025(Conflict2024-06-15-14-28-25).pdf',
 'VTU21511_Academic Fee - Summer Semester - 2024-2025.pdf',
 'snake.pdf',
 'Untitled Diagram.drawio',
 'Major project ',
 'Predictive Inventory Management in Logistics Using Machine Learning(3) (1).pdf',
 'Predictive Inventory Management in Logistics Using Machine Learning(3).pdf',
 'BK charan asish (3).pdf',
 'BK charan asish (2).pdf',
 'BK charan asish (1).pdf',
 'BK charan asish.pdf',
 'IMG_20241213_111646-compressed.jpeg',
 'CHARAN_ASISH[1].docx',
 'Classroom',
 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (3).pdf',
 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (2).pdf',
 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (1).pdf',
 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2).pdf',
 'self_introduction.mp4',
 'Colab Notebooks',
 'ECG_DATA',
 'ecg_model.h5',
 'ecg_model_best.keras',
 'ecg_model_final.keras',
 'ecg_model_final_tl.keras',
 'ecg_model_

In [16]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/drive/MyDrive/ecg_model_final_scratch.keras")
print("Model loaded successfully!")


Model loaded successfully!


In [22]:
import tensorflow as tf
import numpy as np

# ----- Load your trained Sequential model -----
model_path = "/content/drive/MyDrive/ecg_model_final_scratch.keras"  # update if needed
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")

# ----- Step 1: Create a new Sequential model for feature extraction -----
# Exclude the last Dense (softmax) layer
feature_extractor = tf.keras.Sequential(model.layers[:-2])  # second-to-last layer output
print("Feature extractor ready!")

# ----- Step 2: Extract features for your dataset -----
all_features = []
all_labels = []

for x_batch, y_batch in train_ds:  # your tf.data.Dataset
    feats = feature_extractor.predict(x_batch)
    all_features.append(feats)
    all_labels.append(y_batch.numpy())

all_features = np.vstack(all_features)
all_labels = np.hstack(all_labels)

print("All features shape:", all_features.shape)
print("All labels shape:", all_labels.shape)


Model loaded successfully!
Feature extractor ready!


NameError: name 'train_ds' is not defined

In [26]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import os
import librosa
import numpy as np

def load_audio_dataset(base_path, n_mfcc=40):
    features = []
    labels = []

    for label_name in os.listdir(base_path):
        folder = os.path.join(base_path, label_name)
        if os.path.isdir(folder):
            label = 0 if label_name == "normal" else 1  # adjust mapping
            for file in os.listdir(folder):
                if file.endswith(".wav"):
                    file_path = os.path.join(folder, file)
                    y, sr = librosa.load(file_path, sr=22050)
                    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
                    mfcc = np.mean(mfcc.T, axis=0)  # mean over time
                    features.append(mfcc)
                    labels.append(label)
    return np.array(features), np.array(labels)

# Paths
train_path = "/content/drive/MyDrive/heart_sound/train"
val_path = "/content/drive/MyDrive/heart_sound/val"

# Load datasets
train_audio_features, train_audio_labels = load_audio_dataset(train_path)
val_audio_features, val_audio_labels = load_audio_dataset(val_path)

print("Train audio features shape:", train_audio_features.shape)
print("Validation audio features shape:", val_audio_features.shape)


Train audio features shape: (3240, 40)
Validation audio features shape: (301, 40)


In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define model
audio_input = layers.Input(shape=(40,))  # 40 MFCCs
x = layers.Dense(128, activation='relu')(audio_input)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
audio_output = layers.Dense(2, activation='softmax')(x)  # normal vs abnormal

audio_model = models.Model(inputs=audio_input, outputs=audio_output)

audio_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

audio_model.summary()


Model: "functional_132"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,634 (53.26 KB)

 Trainable params: 13,634 (53.26 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = audio_model.fit(
    train_audio_features, train_audio_labels,
    validation_data=(val_audio_features, val_audio_labels),
    epochs=30,
    batch_size=32
)


Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9217 - loss: 2.3819 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [32]:
from tensorflow.keras import Model

# Create feature extractor (second to last dense layer)
feature_extractor_audio = Model(
    inputs=audio_model.input,
    outputs=audio_model.layers[-2].output
)

# Extract embeddings
train_audio_embeddings = feature_extractor_audio.predict(train_audio_features)
val_audio_embeddings = feature_extractor_audio.predict(val_audio_features)

print("Train audio embeddings shape:", train_audio_embeddings.shape)
print("Validation audio embeddings shape:", val_audio_embeddings.shape)


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Train audio embeddings shape: (3240, 64)
Validation audio embeddings shape: (301, 64)


In [38]:
import os

# List all files in your MyDrive
path = "/content/drive/MyDrive"
print(os.listdir(path))


["I am sharing 'Asish Asish' with you", 'peddamma.mp4', 'VTU21511_Academic Fee - Summer Semester - 2024-2025(Conflict2024-06-15-14-28-25).pdf', 'VTU21511_Academic Fee - Summer Semester - 2024-2025.pdf', 'snake.pdf', 'Untitled Diagram.drawio', 'Major project ', 'Predictive Inventory Management in Logistics Using Machine Learning(3) (1).pdf', 'Predictive Inventory Management in Logistics Using Machine Learning(3).pdf', 'BK charan asish (3).pdf', 'BK charan asish (2).pdf', 'BK charan asish (1).pdf', 'BK charan asish.pdf', 'IMG_20241213_111646-compressed.jpeg', 'CHARAN_ASISH[1].docx', 'Classroom', 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (3).pdf', 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (2).pdf', 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2) (1).pdf', 'BK-CHARAN-ASISH-FlowCV-Resume-20250509 (2).pdf', 'self_introduction.mp4', 'Colab Notebooks', 'ECG_DATA', 'ecg_model.h5', 'ecg_model_best.keras', 'ecg_model_final.keras', 'ecg_model_final_tl.keras', 'ecg_model_best_tl.keras', 'ecg_model_

In [39]:
import pandas as pd

# Load Excel file
df = pd.read_excel("/content/drive/MyDrive/cardio_train.xlsx")  # change name if different

print("Shape:", df.shape)
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cardio_train.xlsx'

In [40]:
import os

print(os.listdir("/content/drive/MyDrive/cardio_train.csv"))


['cardio_train.csv']


In [41]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/cardio_train.csv/cardio_train.csv")

print("Shape:", df.shape)
print(df.head())


Shape: (70000, 1)
  id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio
0              0;18393;2;168;62.0;110;80;1;1;0;0;1;0                               
1              1;20228;1;156;85.0;140;90;3;1;0;0;1;1                               
2              2;18857;1;165;64.0;130;70;3;1;0;0;0;1                               
3             3;17623;2;169;82.0;150;100;1;1;0;0;1;1                               
4              4;17474;1;156;56.0;100;60;1;1;0;0;0;0                               


In [43]:
# Drop label column to get features
X = df.drop("label", axis=1)
y = df["label"]

print("Features shape:", X.shape)
print("Labels shape:", y.shape)


KeyError: "['label'] not found in axis"

In [45]:
import pandas as pd

# Reload correctly with semicolon delimiter
df = pd.read_csv("/content/drive/MyDrive/cardio_train.csv/cardio_train.csv", sep=";")

print("Shape:", df.shape)
print(df.head())
print(df.columns)


Shape: (70000, 13)
   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  
0     0       1       0  
1     0       1       1  
2     0       0       1  
3     0       1       1  
4     0       0       0  
Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')


In [46]:
# Drop unnecessary columns
X = df.drop(['id', 'cardio'], axis=1)  # 'id' is not needed
y = df['cardio']  # target label

print("Features shape:", X.shape)
print("Labels shape:", y.shape)


Features shape: (70000, 11)
Labels shape: (70000,)


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data (80% train, 20% val)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalize numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)


X_train shape: (56000, 11)
X_val shape: (14000, 11)


In [48]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define MLP model
tabular_input = tf.keras.Input(shape=(X_train.shape[1],), name="Tabular_Input")
x = layers.Dense(64, activation="relu")(tabular_input)
x = layers.Dropout(0.3)(x)
x = layers.Dense(32, activation="relu")(x)
tabular_output = layers.Dense(1, activation="sigmoid")(x)  # binary output

tabular_model = models.Model(inputs=tabular_input, outputs=tabular_output)

tabular_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
tabular_model.summary()

# Train
history = tabular_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32
)


Model: "functional_135"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Tabular_Input (InputLayer)      │ (None, 11)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6434 - loss: 0.6324 - val_accuracy: 0.7176 - val_loss: 0.5721
Epoch 2/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7184 - loss: 0.5752 - val_accuracy: 0.7259 - val_loss: 0.5574
Epoch 3/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7236 - loss: 0.5657 - val_accuracy: 0.7273 - val_loss: 0.5505
Epoch 4/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7328 - loss: 0.5506 - val_accuracy: 0.7280 - val_loss: 0.5480
Epoch 5/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7286 - loss: 0.5482 - val_accuracy: 0.7317 - val_loss: 0.5476
Epoch 6/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7310 - loss: 0.5475 - val_accuracy: 0.7307 - val_loss: 0.5490
Epoch 7/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7356 - loss: 0.5444 - val_accuracy: 0.7312 - val_loss: 0.5452
Epoch 8/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7314 - loss: 0.5464 - 

NameError: name 'all_features_val' is not defined